In [61]:
import json
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

In [62]:
# Functie care curata textul: il aduce la lowercase si elimina caractere speciale

def preprocess(texts):
    return [re.sub(r"[^\w\s]", "", text.lower()) for text in texts]

In [63]:
# Incarca datele si transforma textele in vectori numerici folosind TF-IDF

def load_and_vectorize_data(file_path):
    df = pd.read_csv(file_path)
    texts = preprocess(df["Text"].tolist())
    labels = df["Sentiment"].tolist()
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(labels)
    vectorizer = TfidfVectorizer(max_features=500)
    X = vectorizer.fit_transform(texts).toarray()
    return X, y, vectorizer

In [64]:
# Creeaza o retea neuronala folosind Keras pentru clasificare binara (pozitiv/negativ)

def build_keras_model(input_size):
    model = Sequential()
    model.add(Input(shape=(input_size,)))  
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Clasifica un mesaj folosind modelul antrenat cu Keras

def predict_sentiment_keras(model, vectorizer, msg):
    msg_processed = [re.sub(r"[^\w\s]", "", msg.lower())]
    msg_vec = vectorizer.transform(msg_processed).toarray()
    pred = model.predict(msg_vec)[0][0]
    return "positive" if pred >= 0.5 else "negative"

In [65]:
# Clasa ce implementeaza manual o retea neuronala simpla cu un strat ascuns

class SimpleANN:
    def __init__(self, input_size, hidden_size, output_size):
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def relu(self, z):
        return np.maximum(0, z)

    def relu_derivative(self, z):
        return (z > 0).astype(float)

    def train(self, X, y, epochs=500, lr=0.1):
        y = y.reshape(-1, 1)
        for _ in range(epochs):
            Z1 = X @ self.W1 + self.b1
            A1 = self.relu(Z1)
            Z2 = A1 @ self.W2 + self.b2
            A2 = self.sigmoid(Z2)

            dZ2 = A2 - y
            dW2 = A1.T @ dZ2 / X.shape[0]
            db2 = np.sum(dZ2, axis=0, keepdims=True) / X.shape[0]

            dA1 = dZ2 @ self.W2.T
            dZ1 = dA1 * self.relu_derivative(Z1)
            dW1 = X.T @ dZ1 / X.shape[0]
            db1 = np.sum(dZ1, axis=0, keepdims=True) / X.shape[0]

            self.W2 -= lr * dW2
            self.b2 -= lr * db2
            self.W1 -= lr * dW1
            self.b1 -= lr * db1

    def predict(self, X):
        A1 = self.relu(X @ self.W1 + self.b1)
        A2 = self.sigmoid(A1 @ self.W2 + self.b2)
        return (A2 > 0.5).astype(int)

def predict_sentiment_simple(model, vectorizer, msg):
    msg_processed = [re.sub(r"[^\w\s]", "", msg.lower())]
    msg_vec = vectorizer.transform(msg_processed).toarray()
    pred = model.predict(msg_vec)[0][0]
    return "positive" if pred else "negative"

In [66]:
# Trimite un mesaj catre Azure Text Analytics pentru a detecta sentimentul

def analyze_sentiment_azure(msg, credential_path="credidentials.json"):
    with open(credential_path) as f:
        creds = json.load(f)
    client = TextAnalyticsClient(
        endpoint=creds["END_POINT"],
        credential=AzureKeyCredential(creds["API_KEY"])
    )
    documents = [msg]
    result = client.analyze_sentiment(documents, show_opinion_mining=True)
    docs = [doc for doc in result if not doc.is_error]
    for doc in docs:
        return {
            "sentiment": doc.sentiment,
            "positive": doc.confidence_scores.positive,
            "neutral": doc.confidence_scores.neutral,
            "negative": doc.confidence_scores.negative
        }

In [ ]:
def rulare():
    print("\n[1] Cerinta: Preprocesarea textului si extragerea caracteristicilor (TF-IDF)...")
    X, y, vectorizer = load_and_vectorize_data("reviews_mixed.csv")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("\n[2] Cerinta: Antrenarea si testarea unui clasificator ANN folosind Keras...")
    keras_model = build_keras_model(X_train.shape[1])
    keras_model.fit(X_train, y_train, epochs=10, batch_size=8, verbose=1)
    keras_acc = keras_model.evaluate(X_test, y_test)[1]
    print(f"Acuratetea pe setul de test (Keras): {keras_acc:.2f} — scorul arata cat de bine a invatat reteaua modelul de antrenament")

    print("\n[3] Cerinta: Antrenarea si testarea unui ANN construit manual (cod propriu)...")
    simple_ann = SimpleANN(X_train.shape[1], 32, 1)
    simple_ann.train(X_train, y_train)
    y_pred_simple = simple_ann.predict(X_test).flatten()
    acc_simple = np.mean(y_pred_simple == y_test)
    print(f"Acuratetea pe setul de test (cod propriu): {acc_simple:.2f} — calculata ca procent de predictii corecte")

    print("\n[4] Cerinta: Clasificarea mesajului dat folosind toate cele 3 metode...")
    mesaj = "By choosing a bike over a car, I’m reducing my environmental footprint. Cycling promotes eco-friendly transportation, and I’m proud to be part of that movement."
    keras_sent = predict_sentiment_keras(keras_model, vectorizer, mesaj)
    simple_sent = predict_sentiment_simple(simple_ann, vectorizer, mesaj)
    print(f"Rezultat (Keras): {keras_sent}")
    print(f"Rezultat (Cod propriu): {simple_sent}")

    print("\n[5] Cerinta: Clasificarea aceluiasi mesaj folosind Azure Text Analytics...")
    azure_result = analyze_sentiment_azure(mesaj)
    print(f"Rezultat (Azure): {azure_result['sentiment']}")
    print(f"Scoruri Azure — Positive={azure_result['positive']:.2f}, Neutral={azure_result['neutral']:.2f}, Negative={azure_result['negative']:.2f}")

rulare()